In [1]:
#import packages
import pandas as pd 
import numpy as np  
import cufflinks as cf  #bridge between pandas and plotly
from chart_studio import plotly as py 
import seaborn as sns 
import plotly.express as px
import matplotlib.pyplot as plt

# in order to see and leverage interactivity of plotyly plots in jupyter the following must be installed
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)
cf.go_offline()
import pandas as pd     
import matplotlib.pyplot as plt 

### Datasets leveraged for this:
I will be leveraging 2 datasets from Kaggle.  The first contains the historical medal counts and event details from 1896 - 2022.  It was published by Joseph Cheng
the url is:  https://www.kaggle.com/datasets/josephcheng123456/olympic-historical-dataset-from-olympediaorg/data

I will be leveraging the following files for this analysis:
Olympic_Athlete_Bio.csv (contains athleteID, name, gender, date of birth,  height, weight, country, country code, etc. for Olympic Atheletes)
Olympic_Athlete_Event_Results.csv( contains Olympic edition, edition_id, country code, sporting event, the event category, athlete name, athlete_id, position rank,  medal)



The second dataset is also from Kaggle.  It contains the medal counts and athlete information for the 2024 Paris Olympics.  It was published by Petro and the url
is: https://www.kaggle.com/datasets/piterfm/paris-2024-olympic-summer-games/data

I will be leveraging the following files for this analysis:
athletes.csv (athlete code, name_TV, country, height, weight)
medalists.csv (medal_date, medal_type, gender, country code, country, nationality, team, team_gender, discipline, event, event type, birth_date, athlete code)


In [2]:
#import historical athlete and event files to reveiw structures and common features and formats in order to concatenate the data.
hist_athletes_df = pd.read_csv('raw_data/Olympic_Athlete_Bio.csv', index_col='athlete_id')
hist_medalists_df = pd.read_csv('raw_data/Olympic_Athlete_Event_Results.csv')

In [3]:
hist_athletes_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 155861 entries, 65649 to 74003
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   name           155861 non-null  object 
 1   sex            155861 non-null  object 
 2   born           151808 non-null  object 
 3   height         105112 non-null  float64
 4   weight         105112 non-null  object 
 5   country        155861 non-null  object 
 6   country_noc    155861 non-null  object 
 7   description    54863 non-null   object 
 8   special_notes  60637 non-null   object 
dtypes: float64(1), object(8)
memory usage: 11.9+ MB


In [4]:
hist_medalists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316834 entries, 0 to 316833
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   edition      316834 non-null  object
 1   edition_id   316834 non-null  int64 
 2   country_noc  316834 non-null  object
 3   sport        316834 non-null  object
 4   event        316834 non-null  object
 5   result_id    316834 non-null  int64 
 6   athlete      316834 non-null  object
 7   athlete_id   316834 non-null  int64 
 8   pos          316834 non-null  object
 9   medal        44687 non-null   object
 10  isTeamSport  316834 non-null  bool  
dtypes: bool(1), int64(3), object(7)
memory usage: 24.5+ MB


In [5]:
hist_athletes_df.head()

,name,sex,born,height,weight,country,country_noc,description,special_notes
athlete_id,,,,,,,,,
65649,Ivanka Bonova,Female,4 April 1949,166.0,55,Bulgaria,BUL,Personal Best: 400 – 53.54 (1980).,NaN
112510,Nataliya Uryadova,Female,15 March 1977,184.0,70,Russian Federation,RUS,NaN,Listed in Olympians Who Won a Medal at the Eur...
114973,Essa Ismail Rashed,Male,14 December 1986,165.0,55,Qatar,QAT,Personal Best: 10000 – 27:20.97 (2006).,Listed in Olympians Who Won a Medal at the Asi...
30359,Péter Boros,Male,12 January 1908,NaN,NaN,Hungary,HUN,"Between 1927 and 1938, Péter Boros competed as...",NaN
50557,Rudolf Piowatý,Male,28 April 1900,NaN,NaN,Czechoslovakia,TCH,Rudolf Piowaty joined the Czechoslovak militar...,NaN


In [6]:
hist_medalists_df.head()

,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
0,1908 Summer Olympics,5,ANZ,Athletics,"100 metres, Men",56265,Ernest Hutcheon,64710,DNS,NaN,False
1,1908 Summer Olympics,5,ANZ,Athletics,"400 metres, Men",56313,Henry Murray,64756,DNS,NaN,False
2,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Harvey Sutton,64808,3 h8 r1/2,NaN,False
3,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Guy Haskins,922519,DNS,NaN,False
4,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Joseph Lynch,64735,DNS,NaN,False


In [7]:
# drop unnecessary columns from historical athlete datafile: description and special_notes
hist_athletes_df = hist_athletes_df.drop(columns=['description', 'special_notes'])
hist_athletes_df

,name,sex,born,height,weight,country,country_noc
athlete_id,,,,,,,
65649,Ivanka Bonova,Female,4 April 1949,166.0,55,Bulgaria,BUL
112510,Nataliya Uryadova,Female,15 March 1977,184.0,70,Russian Federation,RUS
114973,Essa Ismail Rashed,Male,14 December 1986,165.0,55,Qatar,QAT
30359,Péter Boros,Male,12 January 1908,NaN,NaN,Hungary,HUN
50557,Rudolf Piowatý,Male,28 April 1900,NaN,NaN,Czechoslovakia,TCH
...,...,...,...,...,...,...,...
23748,Todd Makler,Male,8 January 1946,183.0,75,United States,USA
58581,Géza Hollósi,Male,2 May 1938,175.0,79,Hungary,HUN
30387,József Keresztessy,Male,19 September 1885,NaN,NaN,Hungary,HUN


In [8]:
# reformat the columns of the historical athlete file
hist_athletes_df['born'] = pd.to_datetime(hist_athletes_df['born'], format='mixed', errors='coerce')
hist_athletes_df['born'] = hist_athletes_df['born'].dt.strftime('%Y/%m/%d')

invalid_dates = hist_athletes_df[hist_athletes_df['born'].isna()]
print("rows with invalid dates: ")
print(len(invalid_dates))

rows with invalid dates: 
4288


#### There are 4,288 records with non-standard or limited information the birthdates of athletes.  I will maintain the records as NaT, in order to not dilute medal counts by gender or country in this analysis.

In [9]:
hist_athletes_df.head()

,name,sex,born,height,weight,country,country_noc
athlete_id,,,,,,,
65649,Ivanka Bonova,Female,1949/04/04,166.0,55,Bulgaria,BUL
112510,Nataliya Uryadova,Female,1977/03/15,184.0,70,Russian Federation,RUS
114973,Essa Ismail Rashed,Male,1986/12/14,165.0,55,Qatar,QAT
30359,Péter Boros,Male,1908/01/12,NaN,NaN,Hungary,HUN
50557,Rudolf Piowatý,Male,1900/04/28,NaN,NaN,Czechoslovakia,TCH


In [10]:
# merge with the historical olympic event results dataframe
merged_historical_df = pd.merge(hist_medalists_df, hist_athletes_df[['sex', 'born', 'height', 'weight', 'country']], on='athlete_id', how='inner')
merged_historical_df.head()

,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport,sex,born,height,weight,country
0,1908 Summer Olympics,5,ANZ,Athletics,"100 metres, Men",56265,Ernest Hutcheon,64710,DNS,NaN,False,Male,1889/06/17,NaN,NaN,Australasia
1,1908 Summer Olympics,5,ANZ,Athletics,"400 metres, Men",56313,Henry Murray,64756,DNS,NaN,False,Male,1886/01/14,NaN,NaN,Australasia
2,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Harvey Sutton,64808,3 h8 r1/2,NaN,False,Male,1882/02/18,NaN,NaN,Australasia
3,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Guy Haskins,922519,DNS,NaN,False,Male,1883/12/23,NaN,NaN,Australasia
4,1908 Summer Olympics,5,ANZ,Athletics,"800 metres, Men",56338,Joseph Lynch,64735,DNS,NaN,False,Male,1878/04/22,NaN,NaN,Australasia


In [11]:
merged_historical_df['medal'].unique()

array([nan, 'Bronze', 'Silver', 'Gold'], dtype=object)

In [12]:
#filter the dataset to medal winners only to be comparable to the Paris 2024 Olympics data
hist_medalists_final_df = merged_historical_df[merged_historical_df['medal'].isin(['Bronze', 'Silver', 'Gold'])]
hist_medalists_final_df = hist_medalists_final_df.drop(columns=['edition_id', 'result_id', 'athlete_id', 'pos'])
hist_medalists_final_df.rename(columns={'sex':'gender'}, inplace=True)
hist_medalists_final_df.head()

,edition,country_noc,sport,event,athlete,medal,isTeamSport,gender,born,height,weight,country
20,1908 Summer Olympics,ANZ,Athletics,"3,500 metres Race Walk, Men",Harry Kerr,Bronze,False,Male,1879/01/28,184.0,76,Australasia
30,1908 Summer Olympics,ANZ,Boxing,"Middleweight, Men",Snowy Baker,Silver,False,Male,1884/02/08,NaN,NaN,Australasia
32,1908 Summer Olympics,ANZ,Rugby,"Rugby, Men",John Barnett,Gold,True,Male,1880/01/19,NaN,NaN,Australasia
33,1908 Summer Olympics,ANZ,Rugby,"Rugby, Men",Phil Carmichael,Gold,True,Male,1884/01/25,NaN,NaN,Australasia
34,1908 Summer Olympics,ANZ,Rugby,"Rugby, Men",Dan Carroll,Gold,True,Male,1887/02/17,NaN,NaN,Australasia United States


In [13]:
hist_medalists_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44686 entries, 20 to 316809
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   edition      44686 non-null  object 
 1   country_noc  44686 non-null  object 
 2   sport        44686 non-null  object 
 3   event        44686 non-null  object 
 4   athlete      44686 non-null  object 
 5   medal        44686 non-null  object 
 6   isTeamSport  44686 non-null  bool   
 7   gender       44686 non-null  object 
 8   born         44375 non-null  object 
 9   height       33370 non-null  float64
 10  weight       33370 non-null  object 
 11  country      44686 non-null  object 
dtypes: bool(1), float64(1), object(10)
memory usage: 4.1+ MB


In [14]:
every_1500th_row = hist_medalists_final_df.iloc[::1500]
every_1500th_row

,edition,country_noc,sport,event,athlete,medal,isTeamSport,gender,born,height,weight,country
20,1908 Summer Olympics,ANZ,Athletics,"3,500 metres Race Walk, Men",Harry Kerr,Bronze,False,Male,1879/01/28,184.0,76,Australasia
21170,2020 Summer Olympics,CUB,Athletics,"Long Jump, Men",Maykel Massó,Bronze,False,Male,1999/05/08,174.0,61,Cuba
30660,2004 Summer Olympics,EGY,Boxing,"Heavyweight, Men",Mohamed El-Sayed,Bronze,False,Male,1973/03/22,185.0,91,Egypt
46721,2020 Summer Olympics,BRA,Surfing,"Shortboard, Men",Ítalo Ferreira,Gold,False,Male,1994/05/06,168.0,68,Brazil
61647,1998 Winter Olympics,AUT,Alpine Skiing,"Combined, Men",Mario Reiter,Gold,False,Male,1970/11/05,176.0,78,Austria
81174,1960 Summer Olympics,AUS,Swimming,"4 × 200 metres Freestyle Relay, Men",Murray Rose,Bronze,True,Male,1939/01/06,185.0,80,Australia
91266,1924 Summer Olympics,FIN,Athletics,"3,000 metres, Team, Men",Ville Ritola,Gold,True,Male,1896/01/18,175.0,66,Finland
103829,2022 Winter Olympics,ROC,Ice Hockey,"Ice Hockey, Men",Vyacheslav Voynov,Silver,True,Male,1990/01/15,180.0,91,ROC Russian Federation
115895,2018 Winter Olympics,CAN,Ice Hockey,"Ice Hockey, Men",Eric O'Dell,Bronze,True,Male,1990/06/21,183.0,95,Canada
127379,2008 Summer Olympics,HUN,Canoe Sprint,"Canadian Doubles, 1,000 metres, Men",György Kozmann,Bronze,True,Male,1978/03/23,179.0,85,Hungary


In [15]:
#import paris medalist file to reveiw structures and common features and formats in order sync up with historical dataset
paris_medalists_df = pd.read_csv('raw_data/Paris_medallists.csv')
paris_athletes_df = pd.read_csv('raw_data/Paris_athletes.csv')

In [16]:
paris_athletes_df.columns

Index(['code', 'name', 'name_short', 'name_tv', 'gender', 'function',
       'country_code', 'country', 'country_long', 'nationality',
       'nationality_full', 'nationality_code', 'height', 'weight',
       'disciplines', 'events', 'birth_date', 'birth_place', 'birth_country',
       'residence_place', 'residence_country', 'nickname', 'hobbies',
       'occupation', 'education', 'family', 'lang', 'coach', 'reason', 'hero',
       'influence', 'philosophy', 'sporting_relatives', 'ritual',
       'other_sports'],
      dtype='object')

In [17]:
# all pertinent athlete data is in the medalist file, except the correctly formatted name, will filter the athelete dataframe for merger with the medalist dataset
columns_paris = ['code', 'name_tv', 'height', 'weight']  
paris_names_df = paris_athletes_df[columns_paris]
paris_names_df.head()

,code,name_tv,height,weight
0,1532872,Artur ALEKSANYAN,0,0.0
1,1532873,Malkhas AMOYAN,0,0.0
2,1532874,Slavik GALSTYAN,0,0.0
3,1532944,Arsen HARUTYUNYAN,0,0.0
4,1532945,Vazgen TEVANYAN,0,0.0


In [18]:
paris_medalists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2315 entries, 0 to 2314
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   medal_date    2315 non-null   object 
 1   medal_type    2315 non-null   object 
 2   medal_code    2314 non-null   float64
 3   name          2312 non-null   object 
 4   gender        2312 non-null   object 
 5   country_code  2312 non-null   object 
 6   country       2312 non-null   object 
 7   country_long  2312 non-null   object 
 8   nationality   2310 non-null   object 
 9   team          1555 non-null   object 
 10  team_gender   1555 non-null   object 
 11  discipline    2315 non-null   object 
 12  event         2315 non-null   object 
 13  event_type    2315 non-null   object 
 14  url_event     2294 non-null   object 
 15  birth_date    2312 non-null   object 
 16  code_athlete  2315 non-null   int64  
 17  code_team     1555 non-null   object 
dtypes: float64(1), int64(1), obj

In [19]:
paris_medalists_df.head()

,medal_date,medal_type,medal_code,name,gender,country_code,country,country_long,nationality,team,team_gender,discipline,event,event_type,url_event,birth_date,code_athlete,code_team
0,2024-07-27,Gold Medal,1.0,EVENEPOEL Remco,Male,BEL,Belgium,Belgium,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,2000-01-25,1903136,NaN
1,2024-07-27,Silver Medal,2.0,GANNA Filippo,Male,ITA,Italy,Italy,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1996-07-25,1923520,NaN
2,2024-07-27,Bronze Medal,3.0,van AERT Wout,Male,BEL,Belgium,Belgium,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1994-09-15,1903147,NaN
3,2024-07-27,Gold Medal,1.0,BROWN Grace,Female,AUS,Australia,Australia,Australia,NaN,NaN,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1992-07-07,1940173,NaN
4,2024-07-27,Silver Medal,2.0,HENDERSON Anna,Female,GBR,Great Britain,Great Britain,Great Britain,NaN,NaN,Cycling Road,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1998-11-14,1912525,NaN


In [20]:
# merge paris medalist dataframe with the paris athlete names dataframe based on athlete code#.
merged_paris_df = pd.merge(paris_medalists_df, paris_names_df, left_on='code_athlete', right_on='code', how='left')
merged_paris_df.head()

,medal_date,medal_type,medal_code,name,gender,country_code,country,country_long,nationality,team,...,event,event_type,url_event,birth_date,code_athlete,code_team,code,name_tv,height,weight
0,2024-07-27,Gold Medal,1.0,EVENEPOEL Remco,Male,BEL,Belgium,Belgium,Belgium,NaN,...,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,2000-01-25,1903136,NaN,1903136.0,Remco EVENEPOEL,0.0,0.0
1,2024-07-27,Silver Medal,2.0,GANNA Filippo,Male,ITA,Italy,Italy,Italy,NaN,...,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1996-07-25,1923520,NaN,1923520.0,Filippo GANNA,0.0,0.0
2,2024-07-27,Bronze Medal,3.0,van AERT Wout,Male,BEL,Belgium,Belgium,Belgium,NaN,...,Men's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/men-s-indi...,1994-09-15,1903147,NaN,1903147.0,Wout van AERT,0.0,0.0
3,2024-07-27,Gold Medal,1.0,BROWN Grace,Female,AUS,Australia,Australia,Australia,NaN,...,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1992-07-07,1940173,NaN,1940173.0,Grace BROWN,0.0,0.0
4,2024-07-27,Silver Medal,2.0,HENDERSON Anna,Female,GBR,Great Britain,Great Britain,Great Britain,NaN,...,Women's Individual Time Trial,ATH,/en/paris-2024/results/cycling-road/women-s-in...,1998-11-14,1912525,NaN,1912525.0,Anna HENDERSON,0.0,0.0


In [21]:
lean_paris_df = merged_paris_df.drop(columns=['medal_date', 'medal_code', 'country_long', 'name', 'nationality', 'url_event', 'code', 'event_type'])
lean_paris_df.head()

,medal_type,gender,country_code,country,team,team_gender,discipline,event,birth_date,code_athlete,code_team,name_tv,height,weight
0,Gold Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,2000-01-25,1903136,NaN,Remco EVENEPOEL,0.0,0.0
1,Silver Medal,Male,ITA,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,1996-07-25,1923520,NaN,Filippo GANNA,0.0,0.0
2,Bronze Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,1994-09-15,1903147,NaN,Wout van AERT,0.0,0.0
3,Gold Medal,Female,AUS,Australia,NaN,NaN,Cycling Road,Women's Individual Time Trial,1992-07-07,1940173,NaN,Grace BROWN,0.0,0.0
4,Silver Medal,Female,GBR,Great Britain,NaN,NaN,Cycling Road,Women's Individual Time Trial,1998-11-14,1912525,NaN,Anna HENDERSON,0.0,0.0


In [22]:
#For sports like Football and Basketball, the Paris 2024 dataset's event column only reflects gender and not the sporting event.  Need to concatenate sport and gender to
#create a column matching the formatting of the historical medals files.

def concatenate_sport_gender(row):
    if row['event'] in ['Women', 'Men']:
        return f"{row['sport']} {row['gender']}"
    return row['sport']

In [23]:
# create a new boolean column isTeam based on the values in team_gender.  If the team_gender is not null, then it is a team sport.
team_values = ['M', 'W', 'X', 'O']
lean_paris_df['isTeamSport'] = lean_paris_df['team_gender'].isin(team_values)
lean_paris_df.head()

,medal_type,gender,country_code,country,team,team_gender,discipline,event,birth_date,code_athlete,code_team,name_tv,height,weight,isTeamSport
0,Gold Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,2000-01-25,1903136,NaN,Remco EVENEPOEL,0.0,0.0,False
1,Silver Medal,Male,ITA,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,1996-07-25,1923520,NaN,Filippo GANNA,0.0,0.0,False
2,Bronze Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,1994-09-15,1903147,NaN,Wout van AERT,0.0,0.0,False
3,Gold Medal,Female,AUS,Australia,NaN,NaN,Cycling Road,Women's Individual Time Trial,1992-07-07,1940173,NaN,Grace BROWN,0.0,0.0,False
4,Silver Medal,Female,GBR,Great Britain,NaN,NaN,Cycling Road,Women's Individual Time Trial,1998-11-14,1912525,NaN,Anna HENDERSON,0.0,0.0,False


In [24]:
#format the medal type to correspond with historical dataset
lean_paris_df['medal'] = lean_paris_df['medal_type'].str.replace(' Medal', '')
lean_paris_df.head()

,medal_type,gender,country_code,country,team,team_gender,discipline,event,birth_date,code_athlete,code_team,name_tv,height,weight,isTeamSport,medal
0,Gold Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,2000-01-25,1903136,NaN,Remco EVENEPOEL,0.0,0.0,False,Gold
1,Silver Medal,Male,ITA,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,1996-07-25,1923520,NaN,Filippo GANNA,0.0,0.0,False,Silver
2,Bronze Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,1994-09-15,1903147,NaN,Wout van AERT,0.0,0.0,False,Bronze
3,Gold Medal,Female,AUS,Australia,NaN,NaN,Cycling Road,Women's Individual Time Trial,1992-07-07,1940173,NaN,Grace BROWN,0.0,0.0,False,Gold
4,Silver Medal,Female,GBR,Great Britain,NaN,NaN,Cycling Road,Women's Individual Time Trial,1998-11-14,1912525,NaN,Anna HENDERSON,0.0,0.0,False,Silver


In [25]:
# reformat the birth_date column to match the formatting in the historical file as well as the name_tv column 
lean_paris_df['born'] = lean_paris_df['birth_date'].str.replace('-','/')
lean_paris_df['athlete'] = lean_paris_df['name_tv'].str.title()
#add a column for the Olympic edition = 2024 Summer Olympics
lean_paris_df['edition'] = '2024 Summer Olympics'
lean_paris_df.head()



,medal_type,gender,country_code,country,team,team_gender,discipline,event,birth_date,code_athlete,code_team,name_tv,height,weight,isTeamSport,medal,born,athlete,edition
0,Gold Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,2000-01-25,1903136,NaN,Remco EVENEPOEL,0.0,0.0,False,Gold,2000/01/25,Remco Evenepoel,2024 Summer Olympics
1,Silver Medal,Male,ITA,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,1996-07-25,1923520,NaN,Filippo GANNA,0.0,0.0,False,Silver,1996/07/25,Filippo Ganna,2024 Summer Olympics
2,Bronze Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,1994-09-15,1903147,NaN,Wout van AERT,0.0,0.0,False,Bronze,1994/09/15,Wout Van Aert,2024 Summer Olympics
3,Gold Medal,Female,AUS,Australia,NaN,NaN,Cycling Road,Women's Individual Time Trial,1992-07-07,1940173,NaN,Grace BROWN,0.0,0.0,False,Gold,1992/07/07,Grace Brown,2024 Summer Olympics
4,Silver Medal,Female,GBR,Great Britain,NaN,NaN,Cycling Road,Women's Individual Time Trial,1998-11-14,1912525,NaN,Anna HENDERSON,0.0,0.0,False,Silver,1998/11/14,Anna Henderson,2024 Summer Olympics


In [26]:
#rename the country_code and discipline columns to match the historical dataset
lean_paris_df.rename(columns={'country_code':'country_noc', 'discipline':'sport'}, inplace=True)
lean_paris_df.head()

,medal_type,gender,country_noc,country,team,team_gender,sport,event,birth_date,code_athlete,code_team,name_tv,height,weight,isTeamSport,medal,born,athlete,edition
0,Gold Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,2000-01-25,1903136,NaN,Remco EVENEPOEL,0.0,0.0,False,Gold,2000/01/25,Remco Evenepoel,2024 Summer Olympics
1,Silver Medal,Male,ITA,Italy,NaN,NaN,Cycling Road,Men's Individual Time Trial,1996-07-25,1923520,NaN,Filippo GANNA,0.0,0.0,False,Silver,1996/07/25,Filippo Ganna,2024 Summer Olympics
2,Bronze Medal,Male,BEL,Belgium,NaN,NaN,Cycling Road,Men's Individual Time Trial,1994-09-15,1903147,NaN,Wout van AERT,0.0,0.0,False,Bronze,1994/09/15,Wout Van Aert,2024 Summer Olympics
3,Gold Medal,Female,AUS,Australia,NaN,NaN,Cycling Road,Women's Individual Time Trial,1992-07-07,1940173,NaN,Grace BROWN,0.0,0.0,False,Gold,1992/07/07,Grace Brown,2024 Summer Olympics
4,Silver Medal,Female,GBR,Great Britain,NaN,NaN,Cycling Road,Women's Individual Time Trial,1998-11-14,1912525,NaN,Anna HENDERSON,0.0,0.0,False,Silver,1998/11/14,Anna Henderson,2024 Summer Olympics


In [27]:
paris_columns = ['edition', 'country_noc', 'sport', 'event', 'athlete', 'medal', 'isTeamSport', 'gender', 'born', 'height', 'weight', 'country']
paris_final_df = lean_paris_df[paris_columns]
paris_final_df.head()

,edition,country_noc,sport,event,athlete,medal,isTeamSport,gender,born,height,weight,country
0,2024 Summer Olympics,BEL,Cycling Road,Men's Individual Time Trial,Remco Evenepoel,Gold,False,Male,2000/01/25,0.0,0.0,Belgium
1,2024 Summer Olympics,ITA,Cycling Road,Men's Individual Time Trial,Filippo Ganna,Silver,False,Male,1996/07/25,0.0,0.0,Italy
2,2024 Summer Olympics,BEL,Cycling Road,Men's Individual Time Trial,Wout Van Aert,Bronze,False,Male,1994/09/15,0.0,0.0,Belgium
3,2024 Summer Olympics,AUS,Cycling Road,Women's Individual Time Trial,Grace Brown,Gold,False,Female,1992/07/07,0.0,0.0,Australia
4,2024 Summer Olympics,GBR,Cycling Road,Women's Individual Time Trial,Anna Henderson,Silver,False,Female,1998/11/14,0.0,0.0,Great Britain


In [28]:
#create the combined dataset for both the historical and Paris Olympics datasets
medalists_df = pd.concat([hist_medalists_final_df, paris_final_df], axis=0)
print(medalists_df.head())
print(medalists_df.tail())

                 edition country_noc      sport                        event  \
20  1908 Summer Olympics         ANZ  Athletics  3,500 metres Race Walk, Men   
30  1908 Summer Olympics         ANZ     Boxing            Middleweight, Men   
32  1908 Summer Olympics         ANZ      Rugby                   Rugby, Men   
33  1908 Summer Olympics         ANZ      Rugby                   Rugby, Men   
34  1908 Summer Olympics         ANZ      Rugby                   Rugby, Men   

            athlete   medal  isTeamSport gender        born  height weight  \
20       Harry Kerr  Bronze        False   Male  1879/01/28   184.0     76   
30      Snowy Baker  Silver        False   Male  1884/02/08     NaN    NaN   
32     John Barnett    Gold         True   Male  1880/01/19     NaN    NaN   
33  Phil Carmichael    Gold         True   Male  1884/01/25     NaN    NaN   
34      Dan Carroll    Gold         True   Male  1887/02/17     NaN    NaN   

                        country  
20              

In [29]:
#add a season and year column to the new dataset
medalists_df['season'] = medalists_df['edition'].str.extract(r'(\bSummer\b|bWinter\b)')
#add and format a year column and add a age column 
medalists_df['year'] = medalists_df['edition'].str.split().str[0]
medalists_df['year'] = pd.to_datetime(medalists_df['year'], format='%Y').dt.year
medalists_df['born'] = pd.to_datetime(medalists_df['born'])
medalists_df['age at Olympics'] = medalists_df['year'] - medalists_df['born'].dt.year
#replace the "weight ranges" in the historical data set with blanks
medalists_df['weight'] = pd.to_numeric(medalists_df['weight'], errors='coerce')
medalists_df.head()

,edition,country_noc,sport,event,athlete,medal,isTeamSport,gender,born,height,weight,country,season,year,age at Olympics
20,1908 Summer Olympics,ANZ,Athletics,"3,500 metres Race Walk, Men",Harry Kerr,Bronze,False,Male,1879-01-28,184.0,76.0,Australasia,Summer,1908,29.0
30,1908 Summer Olympics,ANZ,Boxing,"Middleweight, Men",Snowy Baker,Silver,False,Male,1884-02-08,NaN,NaN,Australasia,Summer,1908,24.0
32,1908 Summer Olympics,ANZ,Rugby,"Rugby, Men",John Barnett,Gold,True,Male,1880-01-19,NaN,NaN,Australasia,Summer,1908,28.0
33,1908 Summer Olympics,ANZ,Rugby,"Rugby, Men",Phil Carmichael,Gold,True,Male,1884-01-25,NaN,NaN,Australasia,Summer,1908,24.0
34,1908 Summer Olympics,ANZ,Rugby,"Rugby, Men",Dan Carroll,Gold,True,Male,1887-02-17,NaN,NaN,Australasia United States,Summer,1908,21.0


In [30]:
#filter to summer olympics only
summer_medalists_df = medalists_df[medalists_df['season']=='Summer']
summer_medalists_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39688 entries, 20 to 2314
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   edition          39688 non-null  object        
 1   country_noc      39685 non-null  object        
 2   sport            39688 non-null  object        
 3   event            39688 non-null  object        
 4   athlete          39685 non-null  object        
 5   medal            39688 non-null  object        
 6   isTeamSport      39688 non-null  bool          
 7   gender           39685 non-null  object        
 8   born             39413 non-null  datetime64[ns]
 9   height           29604 non-null  float64       
 10  weight           29279 non-null  float64       
 11  country          39685 non-null  object        
 12  season           39688 non-null  object        
 13  year             39688 non-null  int32         
 14  age at Olympics  39413 non-null  float64   

In [31]:
summer_medalists_df.describe()

,born,height,weight,year,age at Olympics
count,39413,29604.000000,29279.000000,39688.000000,39413.000000
mean,1952-09-06 20:27:47.876081472,169.634813,68.109362,1978.061480,26.286758
min,1828-10-25 00:00:00,0.000000,0.000000,1896.000000,11.000000
25%,1929-12-30 00:00:00,169.000000,60.000000,1956.000000,22.000000
50%,1962-08-05 00:00:00,178.000000,71.000000,1988.000000,25.000000
75%,1982-04-23 00:00:00,185.000000,82.000000,2008.000000,29.000000
max,2010-05-12 00:00:00,223.000000,182.000000,2024.000000,73.000000
std,NaN,39.454863,24.395184,36.013266,5.943254


In [32]:
#update names for China and Russia
replacement_dict = {
    "People's Republic of China" : "China",
    "Soviet Union" : "Russia",
    "Russian Federation" : "Russia"
}
summer_medalists_df['country'] = summer_medalists_df['country'].replace(replacement_dict)
summer_medalists_df['sport'] = summer_medalists_df['sport'].replace("Athletics", "Track & Field")

In [33]:
#correct the missing sport in the event columns for team sports in the Paris 2024 medalist dataset
#summer_medalists_df['event'] = summer_medalists_df.apply(lambda row: f"{row['sport']}, {row['event']}" if row['event'] in ['Women', 'Men'] else row['event'], axis=1) 
#summer_medalists_df['event'] = summer_medalists_df.apply(lambda row: f"{row['sport']}, {row['event']}" if row['year']==2024 else row['event'], axis=1) 

def update_event(row):
    if row['year']==2024:
        return f"{row['sport']}, {row['event']}"
    elif row['year']!=2024 and row['sport'] not in ['Basketball', 'Football']:
        return f"{row['sport']}, {row['event']}"
    else:
        return row['event']

summer_medalists_df['event'] = summer_medalists_df.apply(update_event, axis=1)

In [34]:
#remove leading spaces in the country column
summer_medalists_df['country'] = summer_medalists_df['country'].str.lstrip()
#remove trailing spaces in the country column
summer_medalists_df['country'] = summer_medalists_df['country'].str.rstrip()

In [35]:
#export scrubbed dataset to csv
summer_medalists_df.to_csv('final_data/summer_medalists.csv', index=False)

In [36]:
#import the official medal tallys by Olympics and country
hist_medals = pd.read_csv('raw_data/Olympic_Games_Medal_Tally.csv')
paris_medals = pd.read_csv('raw_data/Paris_medals_total.csv')

In [37]:
hist_medals.head()

,edition,edition_id,year,country,country_noc,gold,silver,bronze,total
0,1896 Summer Olympics,1,1896,United States,USA,11,7,2,20
1,1896 Summer Olympics,1,1896,Greece,GRE,10,18,19,47
2,1896 Summer Olympics,1,1896,Germany,GER,6,5,2,13
3,1896 Summer Olympics,1,1896,France,FRA,5,4,2,11
4,1896 Summer Olympics,1,1896,Great Britain,GBR,2,3,2,7


In [38]:
#drop edition id column
hist_medals = hist_medals.drop(columns=['edition_id'])

In [39]:
paris_medals.head()

,country_code,country,country_long,Gold Medal,Silver Medal,Bronze Medal,Total
0,USA,United States,United States of America,40,44,42,126
1,CHN,China,People's Republic of China,40,27,24,91
2,JPN,Japan,Japan,20,12,13,45
3,AUS,Australia,Australia,18,19,16,53
4,FRA,France,France,16,26,22,64


In [40]:
hist_medals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1807 entries, 0 to 1806
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   edition      1807 non-null   object
 1   year         1807 non-null   int64 
 2   country      1807 non-null   object
 3   country_noc  1807 non-null   object
 4   gold         1807 non-null   int64 
 5   silver       1807 non-null   int64 
 6   bronze       1807 non-null   int64 
 7   total        1807 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 113.1+ KB


In [41]:
paris_medals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_code  92 non-null     object
 1   country       92 non-null     object
 2   country_long  92 non-null     object
 3   Gold Medal    92 non-null     int64 
 4   Silver Medal  92 non-null     int64 
 5   Bronze Medal  92 non-null     int64 
 6   Total         92 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 5.2+ KB


We need to synchronize the columns between the two datasets prior to merging them.


In [42]:
#extract the season from the Olympics edition in order to filter to Summer Olympics only.
hist_medals['season'] = hist_medals['edition'].str.extract(r'(\bSummer\b|bWinter\b)', expand=False)


In [43]:
hist_medals.columns

Index(['edition', 'year', 'country', 'country_noc', 'gold', 'silver', 'bronze',
       'total', 'season'],
      dtype='object')

In [44]:
# add the missing columns to the paris dataframe
paris_medals['season'] = "Summer"
paris_medals['year'] = 2024
paris_medals['edition'] = '2024 Summer Olympics'
#rename columns to match the hist_medals dataframe
paris_medals = paris_medals.rename(columns={'country_code':'country_noc', 'Gold Medal': 'gold', 'Silver Medal' : 'silver', 'Bronze Medal': 'bronze', 'Total': 'total' })
paris_medals = paris_medals.drop(columns=['country_long'])
paris_medals.head()

,country_noc,country,gold,silver,bronze,total,season,year,edition
0,USA,United States,40,44,42,126,Summer,2024,2024 Summer Olympics
1,CHN,China,40,27,24,91,Summer,2024,2024 Summer Olympics
2,JPN,Japan,20,12,13,45,Summer,2024,2024 Summer Olympics
3,AUS,Australia,18,19,16,53,Summer,2024,2024 Summer Olympics
4,FRA,France,16,26,22,64,Summer,2024,2024 Summer Olympics


In [45]:
#reorder the columns of the paris_medals
new_order = ['edition', 'year', 'country', 'country_noc', 'gold', 'silver', 'bronze','total', 'season']
paris_medals = paris_medals[new_order]
paris_medals.head()

,edition,year,country,country_noc,gold,silver,bronze,total,season
0,2024 Summer Olympics,2024,United States,USA,40,44,42,126,Summer
1,2024 Summer Olympics,2024,China,CHN,40,27,24,91,Summer
2,2024 Summer Olympics,2024,Japan,JPN,20,12,13,45,Summer
3,2024 Summer Olympics,2024,Australia,AUS,18,19,16,53,Summer
4,2024 Summer Olympics,2024,France,FRA,16,26,22,64,Summer


In [46]:
#create the combined dataset for both the historical and Paris Olympics medal tallys
summer_medals = pd.concat([hist_medals, paris_medals], axis=0)
summer_medals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1899 entries, 0 to 91
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   edition      1899 non-null   object
 1   year         1899 non-null   int64 
 2   country      1899 non-null   object
 3   country_noc  1899 non-null   object
 4   gold         1899 non-null   int64 
 5   silver       1899 non-null   int64 
 6   bronze       1899 non-null   int64 
 7   total        1899 non-null   int64 
 8   season       1435 non-null   object
dtypes: int64(5), object(4)
memory usage: 148.4+ KB


In [47]:
#filter to only show summer olympics
summer_df = summer_medals[summer_medals['season'] == 'Summer']
summer_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1435 entries, 0 to 91
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   edition      1435 non-null   object
 1   year         1435 non-null   int64 
 2   country      1435 non-null   object
 3   country_noc  1435 non-null   object
 4   gold         1435 non-null   int64 
 5   silver       1435 non-null   int64 
 6   bronze       1435 non-null   int64 
 7   total        1435 non-null   int64 
 8   season       1435 non-null   object
dtypes: int64(5), object(4)
memory usage: 112.1+ KB


In [48]:
#update names for China and Russia
replacement_dict = {
    "People's Republic of China" : "China",
    "Soviet Union" : "Russia",
    "Russian Federation" : "Russia"
}
summer_df['country'] = summer_df['country'].replace(replacement_dict)

In [49]:
#remove leading spaces in the country column
summer_df['country'] = summer_df['country'].str.lstrip()
#remove trailing spaces in the country column
summer_df['country'] = summer_df['country'].str.rstrip()

In [50]:
#export scrubbed dataset to excel
summer_df.to_excel('final_data/summer_medals.xlsx', index=False)